## Import data and save it as json files


In [1]:
import pandas as pd
import os
import csv
import ast

import json
import requests
from time import sleep
api_key = '1bb6cfd646261b1acda61748ca2bb5a7'

In [2]:
show_id = 253
url = 'http://api.themoviedb.org/3'
search_type = f'/tv/{show_id}'

star_trek_tos = requests.get(url + search_type + "?api_key=" + api_key).json()

In [5]:
url = 'http://api.themoviedb.org/3'
search_type = f'/tv/{show_id}/credits'

dat = requests.get(url + search_type + "?api_key=" + api_key).json()
star_trek_tos['credits'] = dat
main_cast = []
main_cast_ids = []
main_cast_id_to_name = {}

for character in dat['cast']:
    main_cast.append(character['name'])
    main_cast_ids.append(character['id'])
    main_cast_id_to_name[character['id']] = character['name']
    
# 'Majel Barrett', 'Bill Blackburn','Frank da Vinci' not really main cast?
main_cast_ids


[1748, 1749, 1750, 1755, 1754, 1752, 1753, 1751, 1755, 1214943, 1214944]

In [6]:
search_type = f'/tv/{show_id}'

dat = requests.get(url + search_type + "?api_key=" + api_key).json()

seasons = {}
seasons = {season['season_number']: season['episode_count'] for season in star_trek_tos['seasons']}
print(seasons[1])

29


In [27]:
guest_list = []
guest_ids = []
guest_stars_id_to_name = {}

for season_num in seasons:
    episode_count = seasons[season_num]
    star_trek_tos[season_num] = {}
    for episode_num in range(1,episode_count):
        print(season_num, ':' , episode_num, end = '\t')
        sleep(.3)
        dat = requests.get(f'https://api.themoviedb.org/3/tv/{show_id}/season/{season_num}/episode/{episode_num}?api_key={api_key}&language=en-US').json()
        
        for star in dat['guest_stars']:
            try:
                if star['id'] not in guest_ids and star['name'] not in main_cast:
                    guest_list.append(star['name'])
                    guest_ids.append(star['id'])
                    guest_stars_id_to_name[star['id']] = star['name']

                    
            except KeyError:
                print('uh-oh', season_num, ':' , episode_num)
        star_trek_tos[season_num][episode_num] = dat
        

with open('star_trek_ost.txt', 'w') as file:
    json.dump(star_trek_tos, file)

print()
print('DONE')

0 : 1	0 : 2	0 : 3	0 : 4	1 : 1	1 : 2	1 : 3	1 : 4	1 : 5	1 : 6	1 : 7	1 : 8	1 : 9	1 : 10	1 : 11	1 : 12	1 : 13	1 : 14	1 : 15	1 : 16	1 : 17	1 : 18	1 : 19	1 : 20	1 : 21	1 : 22	1 : 23	1 : 24	1 : 25	1 : 26	1 : 27	1 : 28	2 : 1	2 : 2	2 : 3	2 : 4	2 : 5	2 : 6	2 : 7	2 : 8	2 : 9	2 : 10	2 : 11	2 : 12	2 : 13	2 : 14	2 : 15	2 : 16	2 : 17	2 : 18	2 : 19	2 : 20	2 : 21	2 : 22	2 : 23	2 : 24	2 : 25	3 : 1	3 : 2	3 : 3	3 : 4	3 : 5	3 : 6	3 : 7	3 : 8	3 : 9	3 : 10	3 : 11	3 : 12	3 : 13	3 : 14	3 : 15	3 : 16	3 : 17	3 : 18	3 : 19	3 : 20	3 : 21	3 : 22	3 : 23	
DONE


In [39]:
#guest_movies format is {'movie': movie['original_title'], 'movie_id': movie['id'], 'guest_names': [name, name, name] , 'guest_ids': [id_num, id_num, id_num], 'count': 3}
guest_movies = {}
guest_stars = {}
count = 0
movie_ids = []

api_key = '1bb6cfd646261b1acda61748ca2bb5a7'

# loops through each guest star [15620]):# 
for index, id_num in enumerate(guest_ids):
    print(f'\n {count} / {len(guest_ids)-1} - {id_num} - {guest_stars_id_to_name[id_num]}')
    flag = True

    url = f'https://api.themoviedb.org/3/person/{id_num}/movie_credits?api_key={api_key}&language=en-US'
    dat = requests.get(url).json()
    movie_count = 0
    #print(json.dumps(dat , indent = 2, sort_keys= True))
    #if True:
    try:
        # each movie for which current guest star was listed as a cast member
        for movie in dat['cast']:
            #print(json.dumps(dat['cast, indent = 2, sort_keys= True))
                
            try:
                if movie['id'] not in guest_movies:
                    movie_url = f'https://api.themoviedb.org/3/movie/{movie["id"]}?api_key={api_key}&language=en-US'
                    movie_credits_url = f'https://api.themoviedb.org/3/movie/{movie["id"]}/credits?api_key={api_key}&language=en-US'
                    
                    movie_count += 1
                    movie_dat = requests.get(movie_url).json()
                    sleep(.3)
                    movie_credits_dat = requests.get(movie_credits_url).json()
                    
                    sleep(.3)
                    print(movie_count, end = '\t')
                    guest_movies[movie['id']] = movie_dat
                    guest_movies[movie['id']]['credits'] = movie_credits_dat
                    guest_movies[movie['id']]['guest_names'] = []
                    guest_movies[movie['id']]['guest_ids'] = []
                    guest_movies[movie['id']]['count'] = 0
                    
                guest_movies[movie['id']]['guest_names'].append(guest_stars_id_to_name[id_num])
                guest_movies[movie['id']]['guest_ids'].append(id_num)
                guest_movies[movie['id']]['count'] += 1
                
            except KeyError:
                print('should not be able to enter this')
            
    except KeyError:
        #exits if there is no data for guest_id
        print(f'\nERROR - {url} \n {id_num} - {guest_stars_id_to_name[id_num]}')       
        try:
            print(dat['message'])
        except:
            pass
        flag = False
        #Both Elizabeth Rogers and Peter Duryea errors fine they each now exist under two ids, one of which is correct the other popped above
        
    if flag:
        url = f'https://api.themoviedb.org/3/person/{id_num}?api_key={api_key}&language=en-US'
        dat2 = requests.get(url).json()
        guest_stars[id_num] = dat2
        guest_stars[id_num]['credits'] = dat
            
    count += 1
#print(guest_movies[movie['id']])
#print(guest_stars)
with open('guest_stars.txt', 'w') as file:
    json.dump(guest_stars, file)
with open('guest_movies.txt', 'w') as file:
    json.dump(guest_movies, file)


 0 / 301 - 83913 - Joseph Mell
1	2	3	4	5	6	7	8	9	
 1 / 301 - 168061 - Clegg Hoyt
1	2	3	4	5	
 2 / 301 - 82510 - Michael Dugan
1	2	3	4	5	6	7	8	9	
 3 / 301 - 14508 - John Hoyt
1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40	41	42	43	44	45	46	47	48	49	50	51	52	53	54	55	56	57	58	59	60	61	62	63	64	65	66	67	68	69	70	71	72	73	74	75	76	77	
 4 / 301 - 1213153 - Malachi Throne
1	2	3	4	5	6	7	
 5 / 301 - 101742 - Leonard Mudie
1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40	41	42	43	44	45	46	
 6 / 301 - 39770 - Adam Roarke
1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	
 7 / 301 - 1214921 - Bob Johnson
1	
 8 / 301 - 153336 - Meg Wyllie
1	2	3	4	5	
 9 / 301 - 90520 - Susan Oliver
1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	
 10 / 301 - 164771 - Peter Duryea
1	2	3	4	
 11 / 301 - 1214891 - Georgia Schmidt
1	
 12 / 301 - 30551 - Jeffrey Hunter
1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	

1	2	3	4	
 225 / 301 - 1768816 - Ed  McReady

 226 / 301 - 1754177 - Irene Kelly
1	
 227 / 301 - 12298 - Roy Jenson
1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	
 228 / 301 - 16028 - William Marshall
1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	
 229 / 301 - 1768817 - Lois Jewell

 230 / 301 - 2782 - Ian Wolfe
1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40	41	42	43	44	45	46	47	48	49	50	51	52	53	54	55	56	57	58	59	60	61	62	63	64	65	66	67	68	69	70	71	72	73	74	75	76	77	78	79	80	81	82	83	84	85	86	87	88	89	
 231 / 301 - 16033 - Rhodes Reason
1	2	3	4	5	6	7	8	
 232 / 301 - 1214919 - Gil Perkins
1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	
 233 / 301 - 16030 - William Smithers
1	2	3	4	
 234 / 301 - 16031 - Logan Ramsey
1	2	3	4	5	6	7	8	
 235 / 301 - 1563106 - Sheila Leighton
1	2	
 236 / 301 - 16047 - Marj Dusay
1	2	3	4	5	6	7	8	9	10	11	
 237 / 301 - 16051 - Joanne Linville
1	2	3	4	5	6	
 238 / 301 - 16052 - Jack Donner
1	2

In [57]:
#main_cast_movies format is {'movie': movie['original_title'], 'movie_id': movie['id'], 'main_cast_names': [name, name, name] , 'main_cast_ids': [id_num, id_num, id_num], 'count': 3}
main_cast_movies = {}
main_cast_stars = {}
count = 0
movie_ids = []

api_key = '1bb6cfd646261b1acda61748ca2bb5a7'

# loops through each main_cast star #[1748]):# 
for index, id_num in enumerate(main_cast_ids):
    print(f'\n {count} / {len(main_cast_ids)-1} - {id_num} - {main_cast_id_to_name[id_num]}')
    flag = True

    url = f'https://api.themoviedb.org/3/person/{id_num}/movie_credits?api_key={api_key}&language=en-US'
    dat = requests.get(url).json()
    movie_count = 0
    #print(json.dumps(dat , indent = 2, sort_keys= True))
    try:
        # each movie for which current main_cast star was listed as a cast member
        for movie in dat['cast']:
            #print(movie['id'])
            try:
                if movie['id'] not in main_cast_movies:
                    movie_url = f'https://api.themoviedb.org/3/movie/{movie["id"]}?api_key={api_key}&language=en-US'
                    movie_credits_url = f'https://api.themoviedb.org/3/movie/{movie["id"]}/credits?api_key={api_key}&language=en-US'
                    
                    movie_count += 1
                    movie_dat = requests.get(movie_url).json()
                    sleep(.3)
                    movie_credits_dat = requests.get(movie_credits_url).json()
                    sleep(.3)
                    print(movie_count, end = '\t')
                    main_cast_movies[movie['id']] = movie_dat
                    main_cast_movies[movie['id']]['credits'] = movie_credits_dat
                    main_cast_movies[movie['id']]['main_cast_names'] = []
                    main_cast_movies[movie['id']]['main_cast_ids'] = []
                    main_cast_movies[movie['id']]['count'] = 0
                    #print('successful creation')
                
                #print(movie['id'])
                #print(json.dumps(main_cast_movies[movie['id']] , indent = 2, sort_keys= True))
                main_cast_movies[movie['id']]['main_cast_names'].append(main_cast_id_to_name[id_num])
                main_cast_movies[movie['id']]['main_cast_ids'].append(id_num)
                main_cast_movies[movie['id']]['count'] += 1
                
            except KeyError:
                print(f'should not be able to enter this, watch out for guest_stars_id_to_name - {movie_url}')
            
    except KeyError:
        #exits if there is no data for main_cast_id
        print(f'\nERROR - {url} \n {id_num} - {main_cast_guest_stars_id_to_name[id_num]}')       
        try:
            print(dat['message'])
        except:
            pass
        flag = False
        #Both Elizabeth Rogers and Peter Duryea errors fine they each now exist under two ids, one of which is correct the other popped above
        
    if flag:
        url = f'https://api.themoviedb.org/3/person/{id_num}?api_key={api_key}&language=en-US'
        dat2 = requests.get(url).json()
        main_cast_stars[id_num] = dat2
        main_cast_stars[id_num]['credits'] = dat
            
    count += 1
#print(main_cast_movies[movie['id']])
#print(main_cast_stars)
with open('main_cast_stars.txt', 'w') as file:
    json.dump(main_cast_stars, file)
with open('main_cast_movies.txt', 'w') as file:
    json.dump(main_cast_movies, file)


 0 / 10 - 1748 - William Shatner
1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40	41	42	43	44	45	46	47	48	49	50	51	52	53	54	55	56	57	58	59	60	61	62	63	64	65	66	67	68	69	70	71	72	73	74	75	76	77	78	79	80	81	82	83	84	85	86	87	88	89	90	91	92	93	94	95	96	97	98	99	100	101	102	103	104	105	106	107	108	109	110	111	112	113	114	115	116	117	118	119	120	121	122	
 1 / 10 - 1749 - Leonard Nimoy
1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40	41	42	43	44	45	46	47	48	49	50	
 2 / 10 - 1750 - DeForest Kelley
1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	
 3 / 10 - 1755 - Majel Barrett
1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	
 4 / 10 - 1754 - Walter Koenig
1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	
 5 / 10 - 1752 - George Takei
1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40	41	42	43	44	45	46	47	48	49	50	51